# Dependencies - Restart runtime after executing next cell


In [1]:
!pip install svgwrite
!pip install svgpathtools
!pip install cssutils
!pip install lpips
!pip install ftfy regex tqdm
!pip install wandb

# install diffvg
!git clone https://github.com/BachiLi/diffvg
%cd diffvg
!git submodule update --init --recursive
!python setup.py install
%cd ..

# install CLIP
!pip install git+https://github.com/openai/CLIP.git

     |████████████████████████████████| 66 kB 1.8 MB/s 
     |████████████████████████████████| 66 kB 3.9 MB/s 
     |████████████████████████████████| 404 kB 14.0 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=e433eea2082692ed67beb6f1f0aa6c614dd8f90c65d2fb3462b68d9e08ba4c60
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ftfy
     |████████████████████████████████| 1.7 MB 14.5 MB/s 
     |████████████████████████████████| 180 kB 65.8 MB/s 
     |████████████████████████████████| 97 kB 6.2 MB/s 
     |████████████████████████████████| 142 kB 69.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=1a4a52a46c1cdb935375959504670f0c352635f7659b609465a1eda8993

In [1]:
import pydiffvg
import torch
import torchvision.models as models
import numpy as np
import random
import lpips
import clip
import wandb
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from subprocess import call
from google.colab import files

# Differentiable Drawing Wrapper

In [19]:
class DifferentiableDrawer():
    def __init__(self, device, width, height, max_stroke_width=2., gamma=1.0, optim=torch.optim.Adam):
        pydiffvg.set_print_timing(False)
        pydiffvg.set_use_gpu(torch.cuda.is_available())
        pydiffvg.set_device(device)
        self.device = device
        self.canvas_width, self.canvas_height = width, height
        self.max_stroke_width, self.gamma = max_stroke_width, gamma
        self.shapes = []
        self.shape_groups = []
        self.points_optim = optim([torch.tensor(0)], lr=1.)
        self.color_optim = optim([torch.tensor(0)], lr=0.05)
        self.width_optim = optim([torch.tensor(0)], lr=0.01)
        self.renderer = pydiffvg.RenderFunction.apply

    def add_shapes(self, n=256, shape="path", pts_range=(1,4)):
        """
        Add shapes to the set of shapes
        The shape parameter should be :
        - path
        - filled_path 
        """
        # Define shapes
        shapes = []
        shape_groups = []
        if shape=="filled_path":
            for i in range(n):
                num_segments = random.randint(pts_range[0], pts_range[1])
                num_control_points = torch.zeros(num_segments, dtype = torch.int32) + 2
                points = []
                p0 = (random.random(), random.random())
                points.append(p0)
                for j in range(num_segments):
                    radius = 0.05
                    p1 = (p0[0] + radius * (random.random() - 0.5), p0[1] + radius * (random.random() - 0.5))
                    p2 = (p1[0] + radius * (random.random() - 0.5), p1[1] + radius * (random.random() - 0.5))
                    p3 = (p2[0] + radius * (random.random() - 0.5), p2[1] + radius * (random.random() - 0.5))
                    points.append(p1)
                    points.append(p2)
                    if j < num_segments - 1:
                        points.append(p3)
                        p0 = p3
                points = torch.tensor(points)
                points[:, 0] *= canvas_width
                points[:, 1] *= canvas_height
                path = pydiffvg.Path(
                    num_control_points = num_control_points,
                    points = points,
                    stroke_width = torch.tensor(1.0),
                    is_closed = True
                )
                shapes.append(path)
                path_group = pydiffvg.ShapeGroup(
                    shape_ids = torch.tensor([len(shapes) - 1]),
                    fill_color = torch.tensor([
                        random.random(),
                        random.random(),
                        random.random(),
                        random.random()
                    ])
                )
                shape_groups.append(path_group)
        elif shape=="path":
            for i in range(n):
                num_segments = random.randint(pts_range[0], pts_range[1])
                num_control_points = torch.zeros(num_segments, dtype = torch.int32) + 2
                points = []
                p0 = (random.random(), random.random())
                points.append(p0)
                for j in range(num_segments):
                    radius = 0.05
                    p1 = (p0[0] + radius * (random.random() - 0.5), p0[1] + radius * (random.random() - 0.5))
                    p2 = (p1[0] + radius * (random.random() - 0.5), p1[1] + radius * (random.random() - 0.5))
                    p3 = (p2[0] + radius * (random.random() - 0.5), p2[1] + radius * (random.random() - 0.5))
                    points.append(p1)
                    points.append(p2)
                    points.append(p3)
                    p0 = p3
                points = torch.tensor(points)
                points[:, 0] *= canvas_width
                points[:, 1] *= canvas_height
                #points = torch.rand(3 * num_segments + 1, 2) * min(canvas_width, canvas_height)
                path = pydiffvg.Path(
                    num_control_points = num_control_points,
                    points = points,
                    stroke_width = torch.tensor(1.0),
                    is_closed = False
                )
                shapes.append(path)
                path_group = pydiffvg.ShapeGroup(
                    shape_ids = torch.tensor([len(shapes) - 1]),
                    fill_color = None,
                    stroke_color = torch.tensor([
                        random.random(),
                        random.random(),
                        random.random(),
                        random.random()
                    ])
                )
                shape_groups.append(path_group)
        # Get parameters
        points_vars = []
        color_vars = []
        stroke_width_vars = []
        for path in shapes:
            path.points.requires_grad = True
            points_vars.append(path.points)
        if shape=="path":
            for path in shapes:
                path.stroke_width.requires_grad = True
                stroke_width_vars.append(path.stroke_width)
        if shape=="filled_path":
            for group in shape_groups:
                group.fill_color.requires_grad = True
                color_vars.append(group.fill_color)
        else:
            for group in shape_groups:
                group.stroke_color.requires_grad = True
                color_vars.append(group.stroke_color)
        
        # Add parameters to optimizer
        points_new_optim = torch.optim.Adam(points_vars, lr=1.0)
        color_new_optim = torch.optim.Adam(color_vars, lr=0.05)
        width_new_optim = torch.optim.Adam(stroke_width_vars, lr=0.001)
        self.points_optim.param_groups += points_new_optim.param_groups
        self.color_optim.param_groups += color_new_optim.param_groups
        self.width_optim.param_groups += width_new_optim.param_groups

        self.shapes += shapes
        self.shape_groups += shape_groups
        
    def zero_grad(self):
        self.points_optim.zero_grad()
        self.color_optim.zero_grad()
        self.width_optim.zero_grad()

    def step(self):
        self.points_optim.step()
        self.color_optim.step()
        self.width_optim.step()
        try:
            for group in self.shape_groups:
                group.fill_color.data.clamp_(0.0, 1.0)
        except AttributeError:
            for path in self.shapes:
                path.stroke_width.data.clamp_(1.0, max_stroke_width)
            for group in self.shape_groups:
                group.stroke_color.data.clamp_(0.0, 1.0)


    def render(self, seed):
        # Forward pass: render the image.
        scene_args = pydiffvg.RenderFunction.serialize_scene(
            self.canvas_width,
            self.canvas_height,
            self.shapes,
            self.shape_groups
        )
        img = self.renderer(
            self.canvas_width, # width
            self.canvas_height, # height
            2,   # num_samples_x
            2,   # num_samples_y
            seed,   # seed
            None,
            *scene_args
        )
        # Compose img with white background
        img = img[:, :, 3:4] * img[:, :, :3] + torch.ones(img.shape[0], img.shape[1], 3, device = self.device) * (1 - img[:, :, 3:4])
        # Save the intermediate render.
        pydiffvg.imwrite(img.cpu(), 'results/painterly_rendering/iter_{}.png'.format(t), gamma=self.gamma)
        img = img[:, :, :3]
        # Convert img from HWC to NCHW
        img = img.unsqueeze(0)
        img = img.permute(0, 3, 1, 2) # NHWC -> NCHW
        return img

# Image Optimization

In [20]:
# Parameters
n_paths = 150
n_iter = 500
max_stroke_width = 4
gamma = 1.0
percep_loss = False # Set True to compare image features instead of L2 loss
use_filled_path = False # Set True to use filled curves instead of simple path
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize target image
target_img = np.array(Image.open("img1.png").convert('RGB').resize((320,180))) 
target = torch.from_numpy(target_img).to(torch.float32) / 255.0
target = target.pow(gamma)
target = target.to(device)
target = target.unsqueeze(0)
target = target.permute(0, 3, 1, 2) # NHWC -> NCHW
canvas_width, canvas_height = target.shape[3], target.shape[2]

# Initialize diffvg and drawer
drawer = DifferentiableDrawer(
    device, 
    width=canvas_width, 
    height=canvas_height, 
    max_stroke_width=max_stroke_width, 
    gamma=gamma, 
    optim=torch.optim.Adam
)

# Add paths
drawer.add_shapes(n_paths, shape="path", pts_range=(2,5))

l2_criterion = torch.nn.MSELoss()
percep_criterion = lpips.LPIPS(net='vgg')

NAME = "L2_Adam[+100paths/100iter]"
wandb.init(
    project="recvis-project",
    name=NAME
)

with tqdm(range(n_iter)) as loop:
    for t in loop:
        # Every 100 iterations, add some paths
        if t in [200,400]:
            drawer.add_shapes(100, shape="path", pts_range=(2,5))

        metrics = {
            'lpips': None,
            'L2': None,
            'loss': None
        }

        drawer.zero_grad() # Zero grad
        img = drawer.render(t) # Forward pass: render the image.

        # Compute losses
        l2_loss = l2_criterion(img, target)
        lpips_loss = percep_criterion(img,target)
        loss = l2_loss

        loss.backward() # Backpropagate the gradients.
        drawer.step() # Take a gradient descent step.

        # Record metrics
        with torch.no_grad():
            metrics['lpips'] = lpips_loss.item()
            metrics['L2'] = l2_loss.item()
            metrics['loss'] = loss.item()
        loop.set_postfix(render_loss=loss.item())
        wandb.log(metrics)

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.7/dist-packages/lpips/weights/v0.1/vgg.pth


L2,█▆▄▃▂▂▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▆▄▃▂▂▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lpips,██▇▇▇▆▆▅▆▅▅▄▄▄▃▃▆▄▃▃▃▂▂▂▄▃▃▂▂▂▂▂▃▃▂▂▁▁▁▁
L2,0.00647
loss,0.00647
lpips,0.33123


100%|██████████| 500/500 [1:01:11<00:00,  7.34s/it, render_loss=0.0108]


# CLIP-Guided Generation

In [ ]:
# Parameters
n_paths = 250
n_iter = 500
max_stroke_width = 20
gamma = 1.0
n_augment = 4 # Number of augmentation per round
percep_loss = False # Set True to compare image features instead of L2 loss
use_filled_path = False # Set True to use filled curves instead of simple path
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load CLIP and target
model, preprocess = clip.load('ViT-B/32', device, jit=False)
model.eval()
target_txt = clip.tokenize("Watercolor snowy mountain").to(device)
with torch.no_grad():
    target_features = model.encode_text(target_txt)
canvas_width, canvas_height = 224, 224
clip_transforms = transforms.Compose([
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073), 
                         (0.26862954, 0.26130258, 0.27577711))
])
augmentation_transforms = transforms.Compose([
    transforms.RandomPerspective(fill=1, p=1, distortion_scale=0.5),
    transforms.RandomResizedCrop(224, scale=(0.7,0.9)),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073), 
                         (0.26862954, 0.26130258, 0.27577711))
])

# Initialize diffvg and drawer
drawer = DifferentiableDrawer(
    device, 
    width=canvas_width, 
    height=canvas_height, 
    max_stroke_width=max_stroke_width, 
    gamma=gamma, 
    optim=torch.optim.Adam
)

# Add paths
drawer.add_shapes(n_paths, shape="path", pts_range=(2,5))

l2_criterion = torch.nn.MSELoss()
percep_criterion = lpips.LPIPS(net='vgg')

NAME = "CLIP"
wandb.init(
    project="recvis-project",
    name=NAME
)

with tqdm(range(n_iter)) as loop:
    for t in loop:
        drawer.zero_grad() # Zero grad
        img = drawer.render(t) # Forward pass: render the image.

        # Compute losses
        loss = 0
        imgs = torch.cat([augmentation_transforms(img) for i in range(n_augment)])
        imgs_features = model.encode_image(imgs)
        for i in range(n_augment):
            loss -= torch.cosine_similarity(target_features, imgs_features[i:i+1], dim=1)

        loss.backward() # Backpropagate the gradients.
        drawer.step() # Take a gradient descent step.

        loop.set_postfix(render_loss=loss.item())
        wandb.log({'loss': loss.item()})

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 68.5MiB/s]


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.7/dist-packages/lpips/weights/v0.1/vgg.pth


L2,█▅▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▅▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lpips,██▇▇▇▆▆▆▆▅▅▅▅▅▅▄▆▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▃▃▂▂▁▁▁▁
L2,0.01076
loss,0.01076
lpips,0.44415


 40%|███▉      | 198/500 [14:48<29:42,  5.90s/it, render_loss=-1.56]

In [21]:
from google.colab import files
!ffmpeg -y -framerate 60 -i results/painterly_rendering/iter_%d.png -vb 40M results/painterly_rendering/out.mp4
files.download('results/painterly_rendering/out.mp4')

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>